In [ ]:
import sagemaker
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker.inputs import TrainingInput
from sagemaker.pytorch import PyTorch

sagemaker_session = sagemaker.Session()


# To use the sagemaker notebook role requires adding some IAM roles to the notebook.
# Use same roles as in pred-park-de-sagemaker-dev notebook
role = sagemaker.get_execution_role()

In [ ]:
# Data must come from S3 (in the same region as the notebook instance is)
region = sagemaker_session.boto_region_name 

# datainput
if region == "eu-west-1":
    seattle = 's3://vwfs-pred-park-irland/input/open_data/seattle/train_data_with_trans_100_with_transaction.csv'
    
elif region == "eu-central-1":
    seattle = 's3://bucket-vwfs-pred-park-global-model-serving-dev/input/open_data/seattle/train_data_with_trans_100_with_transaction.csv'

else:
    raise NotImplementedError("Region must be eu-west-1 or eu-central-1")

In [ ]:
# Define range for hyperparameters that thaat should be tuned
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.00001, 0.01, scaling_type='Logarithmic'),
    #"weight_decay": ContinuousParameter(0.001, 0.01, scaling_type='Linear'),
    "batch-size": CategoricalParameter([16, 32, 64, 128]),
    "optimizer":CategoricalParameter(["adam", "momentum"]),
    "include_pbp": CategoricalParameter([1,0]),
    "use_batchnorm": CategoricalParameter([1,0]),
    #"loss_func": CategoricalParameter(["sigmoid", "logistic"]),
    "hidden_dim":CategoricalParameter([16,32,64]),
    'lambda_1': ContinuousParameter(0.001, 1, scaling_type='Logarithmic'),
    'lambda_2': ContinuousParameter(0.1, 100, scaling_type='Logarithmic'),
    "output_dim": CategoricalParameter([8,16,32,64]) # 19 features,
}

In [ ]:
# IMPORTANT: Per default, each instance type can only be launched 2x, so every tuning job should run in a different instance type
# instance_list = ["ml.g4dn.xlarge", "ml.g4dn.2xlarge", "ml.g4dn.4xlarge", "ml.g4dn.8xlarge"]
# digit_tasks = ["3vs5", "8vs3"]
# pi_list = [1, 5, 10, 20]

In [ ]:
# tuning params needs to be in the main file's parser, as input to the parser
instance_list = ["ml.g4dn.xlarge"]

# params which is fixed, other than the default of the main file param
fixed_params = {
                "sm_mode": 1,
                "n_experiments": 3, # no. of experiments, eg. tuner jobs -> call main file -> no. experiments -> no. of epoch
                "max_epoch": 200,
                "patience": 10,
                "early_stop": 1,
                "source_only": 0,
                "area_cv": 1
                }
# outocome: one tuning and a few training  # source_only
job_name = f'Parking-with-adaptat'

# define training job-entry point
estimator = PyTorch(
    entry_point="main.py",  # this script will be executed with above args
    role=role,
    source_dir='/home/ec2-user/SageMaker/mobility-predpark-global-ML/research/Discriminative-Feature-Alignment/Parking_SimpleDA',  # the whole directly will be uploaded to docker container
    framework_version="1.4.0",
    py_version="py3",
    instance_count=1,
    instance_type=instance_list[0],  # new instance type for each tuning job
    hyperparameters=fixed_params
)
# docker container will be launched, copy the whole root(so remove the files which are not needed) and check torch.save code
# when docker started, pip install the requirements.txt

# scrape target metric from Cloudwatch Logs (print/logging command)
# metrix to be optimized
objective_metric_name = "Matthew" #naming
objective_type = "Maximize" #direction 
metric_definitions = [{"Name": "Matthew", "Regex": "Average Matthew: (.*?);"}] #maybe star with AUC
# regex

In [ ]:
#estimator.fit({'seattle': seattle}, wait=True)

In [ ]:
"""
1. prep instance (prepaing instances take 3, 4 min usu)
2. download training image (pre-defined dockers images, framework_version="1.4.0")
3. run all the pip installs for requirements 
 "/opt/conda/bin/python3.6 main.py --early 1 --max_epoch 10 --n_experiments 1 --patience 10 --sm_mode 1" fixed params
"""

In [ ]:
# define tuning job around the estimator: each this will launch max 200 training jobs
# HyperparameterTuner wrapp around the estimator
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=100,  #eg. we have tried 250(with 1 experiments), and 100(with 3 experiments)
    max_parallel_jobs=3, # trade off between trainign time and max, not too many parallel, might affect bayesian, each tuner job, will launch one instance
    objective_type=objective_type,
    base_tuning_job_name=job_name,
    strategy='Bayesian', # check
    early_stopping_type='Auto'
)

# start the tuning job, this seattle here looks at the path of the s3 bucket and then put into the opt/ML
tuner.fit({'seattle': seattle}, wait=False) # wait, only for notebook see results or not